In [ ]:
### Automl for daily forecasting

In [48]:
# import packages
import numpy as np
import pandas as pd
import statsmodels as sm
from matplotlib import pyplot as plt
## mlflow integration for model experimentation
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from mlflow.models.signature import infer_signature
import shap
import seaborn as sns
# from pylab import rcParams
# from sklearn.linear_model import LinearRegression
# from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
# from statsmodels.tsa import api as smt
# from statsmodels.tsa.seasonal import seasonal_decompose
# from statsmodels.tsa.stattools import adfuller
import matplotlib as mpl
import warnings
from pprint import pprint
warnings.filterwarnings("ignore")
#modeling packages
#from tqdm import tqdm
#from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing
import metric
from metric import evaluate
#complex amra model
# from statsmodels.tsa.ar_model import AR
# from statsmodels.tsa.arima_model import ARIMA, ARMA
# from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing
# pm model 
# import six
# import sys
# sys.modules['sklearn.externals.six'] = six
# import mlrose
# import joblib
# sys.modules['sklearn.externals.joblib'] = joblib
# import pmdarima as pm
# from statsmodels.tsa.statespace.sarimax import SARIMAX
#ensemble modeling
import lightgbm as lgbm
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer


import plots 
from plots import plot_autocorrelation_grid, plot_grid
import eng_feature 
from eng_feature import check_df,lag_features,roll_mean_features,ewm_features,create_time_features

In [2]:
# stat forecaster use
import random
import time
from multiprocessing import cpu_count, Pool # for prophet
from itertools import product
import matplotlib.pyplot as plt
from statsforecast.core import StatsForecast
from statsforecast.models import auto_arima
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.model_selection import ParameterGrid

from statsforecast.models import (
    adida, 
    croston_classic, 
    croston_sba, 
    croston_optimized,
    historic_average,
    imapa,
    naive,
    random_walk_with_drift, 
    seasonal_exponential_smoothing,
    seasonal_naive, 
    seasonal_window_average,
    ses, 
    tsb,
    window_average
)


In [3]:
date_column ='EffectiveDate'
file_location = 'FleetForecasting_Top100ProductSubCategory_WithIHSData_Weather_BYDay_V2.csv'
input_data_raw=pd.read_csv(file_location ,sep ='|',parse_dates=[date_column])
input_data_copy = input_data_raw.copy()

In [4]:
input_data_copy.head()

,DRKey,Division,EffectiveDate,ProductCategory_Nbl,ProductCategory_Desc,OnRent,QtyOwned,Rental,IHSRentalMarket,AvgTemp,AvgPrcp,Region,RegionName
0,103-1060,103,2016-01-01,1060,CONCRETE FLOOR GRINDERS ...,0,16,1946.0,1.982141,33.633333,0.001666,1,REGION 01
1,103-1060,103,2016-01-03,1060,CONCRETE FLOOR GRINDERS ...,0,16,1946.0,1.982141,33.566666,0.015000,1,REGION 01
2,103-1060,103,2016-01-04,1060,CONCRETE FLOOR GRINDERS ...,0,16,1946.0,1.982141,18.466666,0.021666,1,REGION 01
3,103-1060,103,2016-01-05,1060,CONCRETE FLOOR GRINDERS ...,0,16,1946.0,1.982141,13.100000,0.006666,1,REGION 01
4,103-1060,103,2016-01-06,1060,CONCRETE FLOOR GRINDERS ...,0,15,1946.0,1.982141,21.550000,0.000000,1,REGION 01


In [5]:
#input_data_copy=input_data_copy[input_data_copy["Division"].isin([502,503,504,510,511])]

In [6]:
input_data_copy.columns

Index(['DRKey', 'Division', 'EffectiveDate', 'ProductCategory_Nbl',
       'ProductCategory_Desc', 'OnRent', 'QtyOwned', 'Rental',
       'IHSRentalMarket', 'AvgTemp', 'AvgPrcp', 'Region', 'RegionName'],
      dtype='object')

In [7]:
input_data_copy.shape

(10435558, 13)

In [8]:
input_data_copy.head()

,DRKey,Division,EffectiveDate,ProductCategory_Nbl,ProductCategory_Desc,OnRent,QtyOwned,Rental,IHSRentalMarket,AvgTemp,AvgPrcp,Region,RegionName
0,103-1060,103,2016-01-01,1060,CONCRETE FLOOR GRINDERS ...,0,16,1946.0,1.982141,33.633333,0.001666,1,REGION 01
1,103-1060,103,2016-01-03,1060,CONCRETE FLOOR GRINDERS ...,0,16,1946.0,1.982141,33.566666,0.015000,1,REGION 01
2,103-1060,103,2016-01-04,1060,CONCRETE FLOOR GRINDERS ...,0,16,1946.0,1.982141,18.466666,0.021666,1,REGION 01
3,103-1060,103,2016-01-05,1060,CONCRETE FLOOR GRINDERS ...,0,16,1946.0,1.982141,13.100000,0.006666,1,REGION 01
4,103-1060,103,2016-01-06,1060,CONCRETE FLOOR GRINDERS ...,0,15,1946.0,1.982141,21.550000,0.000000,1,REGION 01


In [9]:
input_data_copy[['EffectiveDate','ProductCategory_Nbl','ProductCategory_Desc','Division','DRKey']].nunique() 
# list out amount of unqiue data for each column - nbl and desc is missing one eq?

EffectiveDate           1926
ProductCategory_Nbl      207
ProductCategory_Desc     206
Division                  60
DRKey                   5490
dtype: int64

In [10]:
input_data_copy.sort_values(by=['EffectiveDate'], ascending=True,inplace=True)

In [11]:
input_data_copy.head()

,DRKey,Division,EffectiveDate,ProductCategory_Nbl,ProductCategory_Desc,OnRent,QtyOwned,Rental,IHSRentalMarket,AvgTemp,AvgPrcp,Region,RegionName
0,103-1060,103,2016-01-01,1060,CONCRETE FLOOR GRINDERS ...,0,16,1946.0,1.982141,33.633333,0.001666,1,REGION 01
8704333,708-7705,708,2016-01-01,7705,SKIDSTEER LOADER 1500-2100LB TRACK ...,21,36,77778.5,4.544840,47.887500,0.075714,12,REGION 12
7710503,608-5305,608,2016-01-01,5305,STRAIGHT BOOM 40' - 49' ...,40,72,83919.0,3.145331,24.322222,0.002500,900,REGION 09
6726467,408-7585,408,2016-01-01,7585,RIDE-ON VIBRATORY ROLLERS SMOOTH DRUM ...,3,7,3660.0,0.854969,50.840000,0.246000,400,REGION 04
6335770,405-2935,405,2016-01-01,2935,FORKLIFT- INDUSTRIAL 10000LB ...,5,6,8750.0,1.088499,39.400000,0.000000,400,REGION 04


In [12]:
input_data_copy[date_column].min(), input_data_copy[date_column].max()  

(Timestamp('2016-01-01 00:00:00'), Timestamp('2022-03-27 00:00:00'))

In [13]:
input_data_copy["OnRent"].describe([0.10, 0.30, 0.50, 0.70, 0.80, 0.90, 0.95, 0.99])

count    1.043556e+07
mean     2.086776e+01
std      4.746920e+01
min      0.000000e+00
10%      1.000000e+00
30%      4.000000e+00
50%      8.000000e+00
70%      1.700000e+01
80%      2.700000e+01
90%      4.800000e+01
95%      7.500000e+01
99%      2.050000e+02
max      1.603000e+03
Name: OnRent, dtype: float64

In [14]:
input_data_copy['clean_time']=pd.to_datetime(input_data_copy[date_column]) # do i need this? 
input_data_copy.sort_values(by=['DRKey',date_column], ascending=False,inplace=True)
input_data_copy['ProductCategory_Desc'] = input_data_copy['ProductCategory_Desc'].str.replace(r"[\"\',< ]", '')
cols = ['ProductCategory_Nbl', 'ProductCategory_Desc']
input_data_copy['eq_nm'] = input_data_copy[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
input_data_copy = lag_features(input_data_copy, [91, 98, 105, 112, 119, 126, 182, 364])
check_df(input_data_copy)
# input_data_copy['OnRent_diff']=input_data_copy.groupby('DRKey')['OnRent'].diff(7)
# input_data_copy['AvgTemp_diff'] = input_data_copy.groupby('DRKey')['AvgTemp'].diff(7)
# input_data_copy['AvgPrcp_diff'] = input_data_copy.groupby('DRKey')['AvgPrcp'].diff(7) 
# establish difference with 1 month period rolling for stationarity

##################### Shape #####################
(10435558, 23)
##################### Types #####################
DRKey                           object
Division                         int64
EffectiveDate           datetime64[ns]
ProductCategory_Nbl              int64
ProductCategory_Desc            object
OnRent                           int64
QtyOwned                         int64
Rental                         float64
IHSRentalMarket                float64
AvgTemp                        float64
AvgPrcp                        float64
Region                           int64
RegionName                      object
clean_time              datetime64[ns]
eq_nm                           object
OnRent_lag_91                  float64
OnRent_lag_98                  float64
OnRent_lag_105                 float64
OnRent_lag_112                 float64
OnRent_lag_119                 float64
OnRent_lag_126                 float64
OnRent_lag_182                 float64
OnRent_lag_364             

In [15]:
input_data_copy = roll_mean_features(input_data_copy, [365, 546])
input_data_copy.tail()

,DRKey,Division,EffectiveDate,ProductCategory_Nbl,ProductCategory_Desc,OnRent,QtyOwned,Rental,IHSRentalMarket,AvgTemp,...,OnRent_lag_91,OnRent_lag_98,OnRent_lag_105,OnRent_lag_112,OnRent_lag_119,OnRent_lag_126,OnRent_lag_182,OnRent_lag_364,OnRent_roll_mean_365,OnRent_roll_mean_546
4,103-1060,103,2016-01-06,1060,CONCRETEFLOORGRINDERS,0,15,1946.0,1.982141,21.550000,...,1.0,1.0,2.0,1.0,4.0,4.0,2.0,3.0,2.910627,3.118357
3,103-1060,103,2016-01-05,1060,CONCRETEFLOORGRINDERS,0,16,1946.0,1.982141,13.100000,...,2.0,0.0,3.0,2.0,4.0,3.0,2.0,3.0,2.902505,3.116109
2,103-1060,103,2016-01-04,1060,CONCRETEFLOORGRINDERS,0,16,1946.0,1.982141,18.466666,...,2.0,0.0,1.0,2.0,2.0,1.0,1.0,3.0,2.894324,3.113835
1,103-1060,103,2016-01-03,1060,CONCRETEFLOORGRINDERS,0,16,1946.0,1.982141,33.566666,...,2.0,0.0,0.0,0.0,2.0,1.0,1.0,3.0,2.886112,3.111547
0,103-1060,103,2016-01-01,1060,CONCRETEFLOORGRINDERS,0,16,1946.0,1.982141,33.633333,...,2.0,2.0,0.0,1.0,1.0,1.0,1.0,1.0,2.877930,3.109239


In [16]:
alphas = [0.95, 0.9, 0.8, 0.7, 0.5]
lags = [91, 98, 105, 112, 180, 270, 365]
input_data_copy = ewm_features(input_data_copy, alphas, lags)
check_df(input_data_copy)

##################### Shape #####################
(10435558, 60)
##################### Types #####################
DRKey                                  object
Division                                int64
EffectiveDate                  datetime64[ns]
ProductCategory_Nbl                     int64
ProductCategory_Desc                   object
OnRent                                  int64
QtyOwned                                int64
Rental                                float64
IHSRentalMarket                       float64
AvgTemp                               float64
AvgPrcp                               float64
Region                                  int64
RegionName                             object
clean_time                     datetime64[ns]
eq_nm                                  object
OnRent_lag_91                         float64
OnRent_lag_98                         float64
OnRent_lag_105                        float64
OnRent_lag_112                        float64
OnRent_lag_

In [17]:
#input_data_copy['OnRent'] = np.log1p(input_data_copy["OnRent"].values)

In [18]:
input_data_copy.groupby(["Division", "eq_nm"]).agg({"OnRent": ["sum", "mean", "median", "std"]})

OnRent                   \
                                                    sum      mean median   
Division eq_nm                                                             
103      1060_CONCRETEFLOORGRINDERS                5173  2.685877    2.0   
         1090_DUSTCOLLECTOR&VACUUMS                4378  2.273105    2.0   
         1150_PORTABLESHOTBLASTERS                 1220  0.649627    0.0   
         1180_CONCRETE&ASPHALTFLOORSAWS-PUSHTYPE   2275  1.181205    1.0   
         1195_CONCRETEANDMASONRYBLADES             8321  4.320353    4.0   
...                                                 ...       ...    ...   
1403     8560_PLATFORMLIFTTRAILERS                 3353  1.866927    1.0   
         8620_WATERTRAILERS                        8405  4.682451    4.0   
         8650_TRENCHCOMPACTOR                      8877  4.609034    4.0   
         8935_WALK-BEHINDTRENCHERS                 2008  1.101481    1.0   
         9295_WOOD&BRUSHCHIPPERS                   1345  0.770332    1.0   

                                                            
                                                       std  
Division eq_nm                                              
103      1060_CONCRETEFLOORGRINDERS               1.922790  
         1090_DUSTCOLLECTOR&VACUUMS               1.642353  
         1150_PORTABLESHOTBLASTERS                0.837155  
         1180_CONCRETE&ASPHALTFLOORSAWS-PUSHTYPE  1.122557  
         1195_CONCRETEANDMASONRYBLADES            2.523175  
...                                                    ...  
1403     8560_PLATFORMLIFTTRAILERS                1.667516  
         8620_WATERTRAILERS                       3.807594  
         8650_TRENCHCOMPACTOR                     2.986950  
         8935_WALK-BEHINDTRENCHERS                1.148240  
         9295_WOOD&BRUSHCHIPPERS                  0.900463  

[5490 rows x 4 columns]

In [19]:
feature_importance = [col for col in input_data_copy.columns if col not in ['RegionName','clean_time','Rental', 'QtyOwned', "ProductCategory_Desc",'ProductCategory_Nbl']]

In [20]:
feature_importance

['DRKey',
 'Division',
 'EffectiveDate',
 'OnRent',
 'IHSRentalMarket',
 'AvgTemp',
 'AvgPrcp',
 'Region',
 'eq_nm',
 'OnRent_lag_91',
 'OnRent_lag_98',
 'OnRent_lag_105',
 'OnRent_lag_112',
 'OnRent_lag_119',
 'OnRent_lag_126',
 'OnRent_lag_182',
 'OnRent_lag_364',
 'OnRent_roll_mean_365',
 'OnRent_roll_mean_546',
 'sales_ewm_alpha_095_lag_91',
 'sales_ewm_alpha_095_lag_98',
 'sales_ewm_alpha_095_lag_105',
 'sales_ewm_alpha_095_lag_112',
 'sales_ewm_alpha_095_lag_180',
 'sales_ewm_alpha_095_lag_270',
 'sales_ewm_alpha_095_lag_365',
 'sales_ewm_alpha_09_lag_91',
 'sales_ewm_alpha_09_lag_98',
 'sales_ewm_alpha_09_lag_105',
 'sales_ewm_alpha_09_lag_112',
 'sales_ewm_alpha_09_lag_180',
 'sales_ewm_alpha_09_lag_270',
 'sales_ewm_alpha_09_lag_365',
 'sales_ewm_alpha_08_lag_91',
 'sales_ewm_alpha_08_lag_98',
 'sales_ewm_alpha_08_lag_105',
 'sales_ewm_alpha_08_lag_112',
 'sales_ewm_alpha_08_lag_180',
 'sales_ewm_alpha_08_lag_270',
 'sales_ewm_alpha_08_lag_365',
 'sales_ewm_alpha_07_lag_91',
 

In [21]:
input_data_copy.head()

,DRKey,Division,EffectiveDate,ProductCategory_Nbl,ProductCategory_Desc,OnRent,QtyOwned,Rental,IHSRentalMarket,AvgTemp,...,sales_ewm_alpha_07_lag_180,sales_ewm_alpha_07_lag_270,sales_ewm_alpha_07_lag_365,sales_ewm_alpha_05_lag_91,sales_ewm_alpha_05_lag_98,sales_ewm_alpha_05_lag_105,sales_ewm_alpha_05_lag_112,sales_ewm_alpha_05_lag_180,sales_ewm_alpha_05_lag_270,sales_ewm_alpha_05_lag_365
10435557,906-9295,906,2022-03-27,9295,WOOD&BRUSHCHIPPERS,4,7,7435.0,4.627335,27.585714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10435556,906-9295,906,2022-03-25,9295,WOOD&BRUSHCHIPPERS,4,7,7435.0,4.627335,41.942857,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10435555,906-9295,906,2022-03-24,9295,WOOD&BRUSHCHIPPERS,4,7,7435.0,4.627335,47.085714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10435554,906-9295,906,2022-03-23,9295,WOOD&BRUSHCHIPPERS,4,7,7435.0,4.627335,50.471428,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10435553,906-9295,906,2022-03-22,9295,WOOD&BRUSHCHIPPERS,2,7,7435.0,4.627335,56.971428,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
#feature_importance=['OnRent',date_column,'eq_nm','Division','IHSRentalMarket','AvgTemp','AvgPrcp'] # not adding differencing just yet
sg_district_elect=input_data_copy.copy()
sg_district_elect_train=sg_district_elect[feature_importance]

In [23]:
sg_district_elect_train.head()

,DRKey,Division,EffectiveDate,OnRent,IHSRentalMarket,AvgTemp,AvgPrcp,Region,eq_nm,OnRent_lag_91,...,sales_ewm_alpha_07_lag_180,sales_ewm_alpha_07_lag_270,sales_ewm_alpha_07_lag_365,sales_ewm_alpha_05_lag_91,sales_ewm_alpha_05_lag_98,sales_ewm_alpha_05_lag_105,sales_ewm_alpha_05_lag_112,sales_ewm_alpha_05_lag_180,sales_ewm_alpha_05_lag_270,sales_ewm_alpha_05_lag_365
10435557,906-9295,906,2022-03-27,4,4.627335,27.585714,0.001666,900,9295_WOOD&BRUSHCHIPPERS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10435556,906-9295,906,2022-03-25,4,4.627335,41.942857,0.081666,900,9295_WOOD&BRUSHCHIPPERS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10435555,906-9295,906,2022-03-24,4,4.627335,47.085714,0.246666,900,9295_WOOD&BRUSHCHIPPERS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10435554,906-9295,906,2022-03-23,4,4.627335,50.471428,0.120000,900,9295_WOOD&BRUSHCHIPPERS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10435553,906-9295,906,2022-03-22,2,4.627335,56.971428,0.000000,900,9295_WOOD&BRUSHCHIPPERS,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
sg_district_elect_train.drop(sg_district_elect_train.loc[sg_district_elect_train[date_column] < '2015-05-01 00:00:00'].index, inplace=True) # prob dont need because data is from 2016
# drop 2020 for covid
sg_district_elect_train.drop(sg_district_elect_train.loc[sg_district_elect_train[date_column] > '2020-02-01 01:00:00'].index, inplace=True)

In [25]:
sg_district_elect_train.set_index(date_column,inplace=True)
sg_district_elect_train.sort_index(inplace=True)
#sg_district_elect_train.dropna(inplace=True)

In [26]:
sg_district_elect_train.head()

,DRKey,Division,OnRent,IHSRentalMarket,AvgTemp,AvgPrcp,Region,eq_nm,OnRent_lag_91,OnRent_lag_98,...,sales_ewm_alpha_07_lag_180,sales_ewm_alpha_07_lag_270,sales_ewm_alpha_07_lag_365,sales_ewm_alpha_05_lag_91,sales_ewm_alpha_05_lag_98,sales_ewm_alpha_05_lag_105,sales_ewm_alpha_05_lag_112,sales_ewm_alpha_05_lag_180,sales_ewm_alpha_05_lag_270,sales_ewm_alpha_05_lag_365
EffectiveDate,,,,,,,,,,,,,,,,,,,,,
2016-01-01,103-1060,103,0,1.982141,33.633333,0.001666,1,1060_CONCRETEFLOORGRINDERS,2.0,2.0,...,1.002447,3.997562,2.994491,2.008203,1.050045,0.405805,0.943013,1.032904,3.968125,2.974270
2016-01-01,302-5140,302,1,1.666213,50.300000,0.195000,300,5140_MANLIFTARTICULATING34-39TOWABLE,6.0,2.0,...,1.383797,6.168281,2.721108,4.868929,2.222965,3.539494,7.055218,1.721989,6.437989,2.585330
2016-01-01,206-5170,206,16,1.354965,38.262500,0.000000,200,5170_MANLIFTARTICULATING40-49ELECTRIC,18.0,11.0,...,18.670884,14.715092,15.306485,14.821312,11.127951,13.377666,16.341304,18.769119,14.616334,15.597877
2016-01-01,108-2065,108,156,2.764309,34.200000,0.056250,1,2065_SCISSORLIFT15-19ELECTRIC,146.0,144.0,...,179.634308,161.308034,149.312823,148.933617,143.502952,136.377800,122.358397,179.355680,162.403954,150.494914
2016-01-01,707-2110,707,42,3.729834,43.028571,0.000000,700,2110_SCISSORLIFT30-33ELECTRIC,57.0,67.0,...,63.673869,60.690602,48.730489,59.555718,67.131951,73.889681,67.879203,64.159448,60.388746,49.009450


In [27]:
sg_training = sg_district_elect_train.copy()
split_date = '2019-5-20'
df_training = sg_training.loc[sg_training.index <= split_date]
df_test = sg_training.loc[sg_training.index > split_date]

## Add time features for date_column

In [28]:
categorical_features =['eq_nm','Division','Region']
target = 'OnRent'

In [29]:
sg_district_elect_train=pd.get_dummies(sg_district_elect_train,columns=categorical_features)

In [30]:
sg_training=sg_district_elect_train
# differencing and dropping first index
#sg_training = sg_training.diff(periods=1)
#sg_training.drop(index=sg_training.index[0], axis=0, inplace=True)
split_date = '2019-5-20'
df_training = sg_training.loc[sg_training.index <= split_date]
df_test = sg_training.loc[sg_training.index > split_date]
print(f"{len(df_training)} months of training data \n {len(df_test)} months of testing data ")

5700464 months of training data 
 1164514 months of testing data 


In [31]:
df_training.index.min(), df_training.index.max()

(Timestamp('2016-01-01 00:00:00'), Timestamp('2019-05-20 00:00:00'))

In [32]:
# include oberservations - list out all the stuff out

In [33]:
df_test.index.min(), df_test.index.max() #8 months of testing out -

(Timestamp('2019-05-21 00:00:00'), Timestamp('2020-01-31 00:00:00'))

In [34]:
X_train_df, y_train = create_time_features(
    df_training, target=target)

X_test_df, y_test = create_time_features(df_test, target=target)

numerical_data = ['hour',
 'dayofweek',
 'quarter',
 'month',
 'year',
 'dayofyear',
 'sin_day',
 'cos_day',
 'dayofmonth',
 'weekofyear']

categorical_data=set(X_train_df.columns)-set(numerical_data)
print(f'I have {len(categorical_data)} of categorical data and {len(numerical_data)} numerical features') # inaccurate have to add ihs and other diff - no point for this fstring
##TODO remove fstring find out easier way to identify numerical data - data values more than 2???

I have 329 of categorical data and 10 numerical features


In [35]:
# ct = ColumnTransformer([
#         ('standard', StandardScaler(),numerical_data) #fit numerical data
#     ], remainder='passthrough')

# X_train=ct.fit_transform(X_train_df)
# X_test=ct.fit_transform(X_test_df) # am I losing information by fitting time? may not use this ..

resultsDict = {}
predictionsDict = {}

In [36]:
X_train_df.head(2)

,DRKey,IHSRentalMarket,AvgTemp,AvgPrcp,OnRent_lag_91,OnRent_lag_98,OnRent_lag_105,OnRent_lag_112,OnRent_lag_119,OnRent_lag_126,...,hour,dayofweek,quarter,month,year,dayofyear,sin_day,cos_day,dayofmonth,weekofyear
EffectiveDate,,,,,,,,,,,,,,,,,,,,,
2016-01-01,103-1060,1.982141,33.633333,0.001666,2.0,2.0,0.0,1.0,1.0,1.0,...,0,4,1,1,2016,1,0.841471,0.540302,1,53
2016-01-01,302-5140,1.666213,50.300000,0.195000,6.0,2.0,3.0,6.0,5.0,8.0,...,0,4,1,1,2016,1,0.841471,0.540302,1,53


In [37]:
X_test_df.head(2)

,DRKey,IHSRentalMarket,AvgTemp,AvgPrcp,OnRent_lag_91,OnRent_lag_98,OnRent_lag_105,OnRent_lag_112,OnRent_lag_119,OnRent_lag_126,...,hour,dayofweek,quarter,month,year,dayofyear,sin_day,cos_day,dayofmonth,weekofyear
EffectiveDate,,,,,,,,,,,,,,,,,,,,,
2019-05-21,405-5320,1.259423,78.466666,0.000000,59.0,68.0,60.0,56.0,61.0,64.0,...,0,1,2,5,2019,141,0.363171,-0.931722,21,21
2019-05-21,904-5140,1.801897,48.762500,0.140833,8.0,5.0,4.0,3.0,4.0,5.0,...,0,1,2,5,2019,141,0.363171,-0.931722,21,21


## Lightgbm Model and Diagnostic Charts

In [38]:
## TODO Cross Validation/ Grid Search parameters for more defined trees
mpl.rcParams['agg.path.chunksize'] = 10000

In [39]:
EXPERIMENT_NAME = 'On_Rent_Time_Series'
mlflow.set_runName = 'lgbm predictions'# doesnt do anything
client = MlflowClient()
experiments = client.list_experiments()
if EXPERIMENT_NAME not in [e.name for e in experiments]:
    mlflow.create_experiment(EXPERIMENT_NAME)

In [66]:
mlflow.set_experiment(EXPERIMENT_NAME)
with mlflow.start_run(run_name='simple_light'):
    lightGBM = lgbm.LGBMRegressor()
    lightGBM.fit(X_train_df.drop(columns='DRKey'), y_train)
    yhat = lightGBM.predict(X_test_df.drop(columns='DRKey'))
    resultsDict['Lightgbm'] = evaluate(df_test.OnRent, yhat)
    predictionsDict['Lightgbm'] = yhat
    mlflow.log_metrics(evaluate(df_test.OnRent, yhat))
    signature = infer_signature(X_train_df.drop(columns='DRKey'), lightGBM.predict(X_test_df.drop(columns='DRKey')))
    mlflow.lightgbm.log_model(lightGBM, "lgbm", signature = signature)
    ## general diagnostics - 
    plt.plot(df_test.OnRent.values, label='Original')
    plt.plot(yhat, color='red', label='lgbm')
    plt.legend()
    plt.savefig('images/AVP.png')
    plt.clf()
    shap_values = shap.TreeExplainer(lightGBM).shap_values(X_train_df.drop(columns='DRKey'))
    shap.summary_plot(shap_values, X_train_df.drop(columns='DRKey'),show=False)
    plt.savefig('images/shap.png',bbox_inches='tight')
    plt.clf()
    feature_imp = pd.DataFrame(sorted(zip(lightGBM.feature_importances_,X_train_df.drop(columns='DRKey').columns)), columns=['Value','Feature'])
    plt.figure(figsize=(20, 10))
    data=feature_imp.sort_values(by="Value", ascending=False).head(30)
    sns.barplot(x="Value", y="Feature", data=data)
    plt.title('LightGBM Features (avg over folds)')
    plt.savefig('images/lgbm_importances-01.png')
    plt.clf()
    mlflow.log_artifacts("images", artifact_path="diagnostics")
mlflow.end_run

<function mlflow.tracking.fluent.end_run(status: str = 'FINISHED') -> None>

<Figure size 576x684 with 0 Axes>

<Figure size 1440x720 with 0 Axes>

## Forecaster for univerariate models 

#### Dont run if you dont have appropriate conda env - will break azure auto ml

In [ ]:
#df_test=df_test[['EffectiveDate','DRKey','lgbm','OnRent']]

In [ ]:
forecaster_data=input_data_copy.copy()

In [ ]:
#forecaster_data.drop(forecaster_data.loc[forecaster_data[date_column] < '2015-05-01 00:00:00'].index, inplace=True) # prob dont need because data is from 2016
forecaster_data.drop(forecaster_data.loc[forecaster_data[date_column] > '2020-02-01 01:00:00'].index, inplace=True)
forecaster_data=forecaster_data[['DRKey','EffectiveDate','OnRent']]

In [ ]:
forecaster_data.rename(columns={'DRKey': 'unique_id'},inplace = True)
forecaster_data.rename(columns={'EffectiveDate': 'ds'},inplace = True)
forecaster_data.rename(columns={'OnRent': 'y'},inplace = True)
date_column = 'ds' #changing column name for stat forecast
df_forecaster_training = forecaster_data.loc[forecaster_data[date_column] <= split_date]
df_forecaster_test = forecaster_data.loc[forecaster_data[date_column] > split_date]

In [ ]:
df_forecaster_training.head()

In [ ]:
n_series = 1
seasonality = 70
models = [
    naive,
    historic_average,
    random_walk_with_drift,
    (seasonal_exponential_smoothing, seasonality, 0.6),
    (seasonal_naive, seasonality),
    auto_arima
]
fcst = StatsForecast(df_forecaster_training.set_index('unique_id'), models=models, freq='D', 
                     n_jobs=min(cpu_count(), n_series))

In [ ]:
init = time.time()
with np.errstate(invalid='ignore'):
    forecasts = fcst.forecast(256)
end = time.time()

time_nixtla = end - init
time_nixtla

In [ ]:
forecasts = forecasts.reset_index()

In [ ]:
df_test_merge=df_test.merge(forecasts, how='left', left_on=['DRKey','EffectiveDate'],right_on=['unique_id', 'ds'])

In [ ]:
df_test_merge.drop(columns =['unique_id','ds'], inplace = True)

In [ ]:
df_test_merge.head()

In [ ]:
error_metric={}
for x in df_test_merge.columns :
    if(x not in ['EffectiveDate','DRKey','OnRent']):
        error_metric[x]=(evaluate(df_test_merge.OnRent, df_test_merge[x]))

In [ ]:
error_metric

In [ ]:
#df_test_merge.to_csv('forecast_lightgbm_arima.csv')